# Now let's do some forward dynamics !!
Forward dynamics is conceptually pretty simple. 
You have a dynamic model that you inject some internal force (mimicking the effect of muscles and you simulate over time what it does.
The main problem is that while it is conceptually very simple, finding useful force values to inject is pretty hard. This is what we investigate in this file.

Your goal will be to find a set of muscle activation that keeps the arm still 

## Load a previously created bioMod file
This first section prepares Python and load a `.bioMod` file as shown in`4-muscular_model`.
Please note that for simplicity, we won't use the full body model in here, so this previous file should be run using `use_upper_limb=True`

In [ ]:
# So first, let's import all the required modules 
import biorbd  # This is the core that will do all the calculation
import numpy
from matplotlib import pyplot
from scipy import optimize, integrate

In [ ]:
# Import and interpret the bioMod of the full body (created by the script `4-muscular_model` with use_upper_limb set to True)
model = biorbd.Model("models/SimpleUpperLimbWithMuscles.bioMod")
target_position = (1.20, )

# Define some useful variables
n_q = model.nbQ()
n_muscles = model.nbMuscles()

## Integration
The core concept of forward dynamics is that you "know" the joint torque (or muscle activation) and you want to infer the implied movement.
To do that, one must integrate over time.
Unfortunately, there is no exact method to do so, and therefore one must fallback on numerical methods.

In this section we are creating different method to integrate, namely Euler integration, RK4 and RK45, and investigate the effect of their choice on the resulting kinematics

In [ ]:
# Let's first copy the function  to convert activation to joint torque from `5-static_optimization`
def muscular_joint_torque(activation, q, qdot):
    states = model.stateSet()  # Get the muscle state set
    for a, state in zip(activation, states):
        state.setActivation(a)  # And fill it with the current value
    return model.muscularJointTorque(states, q, qdot).to_array()

# Now, let's define a function that takes in a position (q), velocities (qdot) and the joint torques (tau)
# and spits out the joint acceleration (qddot) that results from this configuration
def forward_dynamics(q, qdot, tau):
    return model.ForwardDynamics(q, qdot, tau).to_array()

# By combining the two previous function we can now get the acceleration resulting from a specific set of muscle activation
def muscular_joint_acceleration(activation, q, qdot):
    tau = muscular_joint_torque(activation, q, qdot)
    return forward_dynamics(q, qdot, tau)

In [ ]:
# We can now test this muscular_joint_acceleration, assuming the target position without velocity and a set of activation to (0.5, 0.5)
q = numpy.array(target_position)
qdot = numpy.zeros(q.shape)
activation = (0.5, 0.5)
print(f"The joint acceleration resulting from activating the muscles at ({activation} while at position ({q}) with velocity ({qdot}) " +
      f"is: {muscular_joint_acceleration(activation, q, qdot)}")

## What's next?
So now we know that the acceleration is not $0m/s/s$ but what does this imply?
This implies that at the next "frame" the arm will have move. 
So in one second the velocity will by equal the the previous number.

Hum... not so fast...
Because as the position changes, so the acceleration because the gravity vector won't act the same on the body.
We must therefore slowly move forward in time. 

The most basic method to do so is the Euler integration method. 
This method simply consists in adding to a given state at time $t$ ($x_t$) the multiplication of the change rate by the time spent ($\Delta t$) like so:
\begin{align}
x_{t+1} = x_t + \dot{x} \Delta t
\end{align}

In [ ]:
# So let's write our own euler integration method
def euler(dt, x, dot_fun, activation):
    return x + dot_fun(x, activation) * dt


# So what is x? x is the state vector. In our case, it is both the position (q) and the velocity (qdot)
# And dot_fun? dot_fun is a function handler that returns the derivative of x. Let's define it now
def x_dot(x, activation):
    # The first part of the state vector is q and the second part is qdot
    q = x[:n_q]
    qdot = x[n_q:]
    
    # The derivative of q is simply qdot
    # The derivative of qdot is given by muscular_joint_acceleration assuming a given set of muscle activation
    xdot = numpy.array((qdot, muscular_joint_acceleration(activation, q, qdot)))[:, 0]
    return xdot    


# Now assuming some initial values, we can perform the integration up to a required final time
def perform_integration(final_time, dt, x_initial, x_dot_fun, activation, integration_fun):
    time_vector = [0.]
    all_x = [x_initial]
    while time_vector[-1] <= final_time:  # As long as we did not get to the final time continue
        time_vector.append(time_vector[-1] + dt)  # The next time is dt later
        all_x.append(integration_fun(dt, x_initial, x_dot_fun, activation))  # Integrate
        x_initial = all_x[-1]  # The next x is the arrival state of the previous integration

    # Format the time vector and the x so they are easier to use
    time_vector = numpy.array(time_vector)
    all_x = numpy.array(all_x).transpose()
    
    return time_vector, all_x

In [ ]:
# To test the function, we just have to set a dt, some initial value and a set of muscle activation of (10% and 0% for Biceps and Triceps respectively
# and to repeat for a number of time

final_time = 2  # Stop at 2 seconds
dt = 0.1
x = numpy.concatenate((target_position, numpy.zeros(n_q)))
activation = numpy.array((0.1, 0.0001))
time_vector_euler, all_x_euler = perform_integration(final_time, dt, x, x_dot, activation, euler)

In [ ]:
# The easiest way to see what happens is to actually animates it 
# We can use bioviz to do so. Again, this comes with the jupyter limitations
import bioviz

# Send the previously loaded model to the vizualizer
viz = bioviz.Viz(loaded_model=model)

# Move the model to a recognizable position (arm raised and knee flexed)
viz.load_movement(all_x_euler[:n_q, :])

# Halt the program so you can admire the movement with the vizualiser. Closing the window should allow to continue
viz.exec()

## What is going on?
So we see that the arm is slowly falling and tries to get back. 
This basically means that $10\%$ activation for the biceps was not enough to keep the arm raised.
You can try to find the actual best solution by changing the `activation` vector.

## What's next?
So now that we have a rudimentary integrator, we can push things a little further. 
Euler integration method is actually very poor. 
It can perform well on very small time step ($\Delta t$) integrated over a small amount of time. 
But this method is highly susceptible to drift and integration error, particularly in non-linear systems.
So a better way to do the integration is to do it using the runge-kutta method.

The runge-kutta (RK) is conceptually the same thing as euler method, but including extra points over the integration period so it corrects itself in the case it is not going in the right direction.
Is this complicated to implement? Let's build our own 4th order runge-kutta (the so-called `rk4`) to see.

In [ ]:
def rk4(dt, x, dot_fun, activation):
    k1 = dot_fun(x, activation)
    k2 = dot_fun(x + dt / 2 * k1, activation)
    k3 = dot_fun(x + dt / 2 * k2, activation)
    k4 = dot_fun(x + dt * k3, activation)
    return x + dt / 6 * (k1 + 2 * k2 + 2 * k3 + k4)

In [ ]:
# And now perform the same integration as before, but integrate using rk4
final_time = 2  # Stop at 1 second
dt = 0.1
x = numpy.concatenate((target_position, numpy.zeros(n_q)))
activation = (0.1, 0.0001)
time_vector_rk4, all_x_rk4 = perform_integration(final_time, dt, x, x_dot, activation, rk4)

In [ ]:
# The easiest way to see what happens is to actually animates it 
# We can use bioviz to do so. Again, this comes with the jupyter limitations
import bioviz

# Send the previously loaded model to the vizualizer
viz = bioviz.Viz(loaded_model=model)

# Move the model to a recognizable position (arm raised and knee flexed)
viz.load_movement(all_x_rk4[:n_q, :])

# Halt the program so you can admire the movement with the vizualiser. Closing the window should allow to continue
viz.exec()

In [ ]:
# And just so you know, rk are pretty common and are generally implemented in linear algebra toolboxes
# Here is an example using scipy and its implementation of rk45 (which would replace the perform optimization from before)
def x_dot_interface(t, x):
    return x_dot(x, activation)

final_time = 2  # Stop at 2 seconds
x = numpy.concatenate((target_position, numpy.zeros(n_q)))
activation = numpy.array((0.1, 0.0001))
time_vector_euler, all_x_euler = perform_integration(final_time, dt, x, x_dot, activation, euler)
results = integrate.solve_ivp(x_dot_interface, [0, final_time], x)
time_vector_rk45 = results.t
all_x_rk45 = results.y

In [ ]:
# We can now compare the two integrations on the same graph
pyplot.figure()
pyplot.plot(time_vector_euler, all_x_euler[0, :])
pyplot.plot(time_vector_rk4, all_x_rk4[0, :])
pyplot.plot(time_vector_rk45, all_x_rk45[0, :])
pyplot.legend(("Euler", "RK4", "RK45"))
pyplot.ylabel("Angle (rad)")
pyplot.xlabel("Time (s)")

## What there is to look at?
The difference between them is quite large. 
When they are performing more or less the same shape, the actual values are wrong, showing how poorly Euler's method behaves.
I invite you to change the $\Delta t$ of the Euler's method to see how small it needs to be similar to RK4. 
Remember that the smaller it is, the more calls `muscular_joint_acceleration` are made.
While for such a small model we don't really mind, for larger models in an optimisation setting this can be the difference between minutes and weeks of optimization time

## Are we done yet?
Last but not least... I told you before your goal would be to find the actual value for the activation so the arm remains static.
You have two choices to do so: first you go by trial and error, or second you let the computer finds. 
Let's do the second using an optimization scheme.

In [ ]:
# Let's define the global parameters of the movement
final_time = 2  # Stop at 2 seconds
dt = 0.1
states = list(target_position) + [0] * n_q  # q and qdot
controls = [0.1, 0.0001]  # muscle activation
initial_guess = numpy.array(states + controls)  # The actual optimization variables sent to the optimizer
integration_method = rk4
x_dot_fun = x_dot

bounds = ((-2*numpy.pi, 2*numpy.pi),) * n_q + ((-20*numpy.pi, 20*numpy.pi),) * n_q + ((0.001, 0.999),) * n_muscles

def lagrange_cost_function(x):
    # Perform the integration over time and compare the position with the target at each integration points (the so-called node)
    states = x[:n_q*2]
    controls = numpy.array(x[n_q*2:])
    t, x_integrated = perform_integration(final_time, dt, states, x_dot_fun, controls, integration_method)
    return numpy.sum((x_integrated[:n_q] - numpy.array(target_position)) ** 2)


results = optimize.minimize(lagrange_cost_function, initial_guess, bounds=bounds)
optimized_states = results.x[:n_q*2]
optimized_controls = results.x[n_q*2:]

In [ ]:
# Using the previous results, let's integrate one last time and graph the results
t, x = perform_integration(final_time, dt, optimized_states, x_dot_fun, optimized_controls, integration_method)
q_optimized = x[:n_q, :]

pyplot.figure()
pyplot.plot(t, q_optimized[0, :])
pyplot.ylim((0, numpy.pi))
pyplot.ylabel("Angle (rad)")
pyplot.xlabel("Time (s)")


## Final thoughts
What we have performed here is a (very) simple version of what is called optimal control, that is finding the controls (for instance muscle activation) that minimizes some criteria and that usually respect some bounds and constraints. 
It is a vast topic and it gets complicated very quickly. 

For those who are interested, the `bioptim` toolbox is an optimal control toolbox based on `biorbd` designed to perform this kind of biomechanics data analyses. 
For instance, one can generate movements _ex nihilo_, find the muscle activations that fits an pre-acquired kinematics, etc. 
The possilibity are vast and we strongly encourage you to explore it.

For now, we are done, and we hope you learn something good!